## Projeto 3 - Sistema de Recomendação com Spark MLLib

### *********** REQUISITO *********** 
Utilize Java JDK 11 e Apache Spark 2.4.2

### *********** DEFINIR O PROBLEMA DE NEGÓCIO *********** 


Neste  projeto,  veremos  como  aplicar  um  algoritmo  de  Sistema  de Recomendação e calcular o affinity score, a fim de descobrir se um produto deve ou não ser recomendado a um usuário.

<strong> O que é um Sistema de recomendação? </strong>
<ul style="list-style-type:square">
  <li>Também chamado de filtros colaborativos.</li>
  <li>Analisa dados passados para compreender comportamentos de pessoas/entidades.</li>
  <li>A recomendação é feita por similaridade de comportamento.</li>
  <li>Recomendação baseada em usuários ou items.</li>
  <li>Algoritmos de Recomendação esperam receber os dados em um formato específico: [user_ID, item_ID, score].</li>
  <li>Score, também chamado rating, indica a preferência de um usuário sobre um item. Podem ser valores booleanos, ratings ou mesmo volume de vendas.</li>
</ul>

In [ ]:
# Importar os pacotes
from pyspark.ml.recommendation import ALS

### *********** CARGA DE DADOS *********** 


O dataset para este projeto será um "dummy data".

In [ ]:
# Criar a Spark Session - usada quando se trabalha com Dataframes no Spark
spSession = SparkSession.builder.master("local").appName("SparkMLLib").getOrCreate()

In [ ]:
# Carregar os dados no formato ALS (user, item, rating)
avaliacaoRDD = sc.textFile("data/user-item.txt")

### *********** ANALISE EXPLORATORIA *********** 

In [ ]:
# Verificar o tipo de objeto
type(avaliacaoRDD)

In [ ]:
# Verificar o tamanho do dataset
avaliacaoRDD.count()

In [ ]:
# Apresentar os dados
avaliacaoRDD.collect()

### *********** LIMPEZA, TRANSFORMAÇÃO, MANIPULAÇÃO DE DADOS *********** 

In [ ]:
# Convertendo as strings
avaliacaoRDD2 = avaliacaoRDD.map(lambda l: l.split(',')).map(lambda l:(int(l[0]), int(l[1]), float(l[2])))

In [ ]:
# Criando um Dataframe
df_avaliacao = spSession.createDataFrame(avaliacaoRDD2, ["user", "item", "rating"])

In [ ]:
df_avaliacao.show()

### *********** CRIAR O MODELO PREDITIVO *********** 

In [ ]:
# ALS = Alternating Least Squares --> Algoritmo para sistema de recomendação, que otimiza a loss function 
# e funciona muito bem em ambientes paralelizados
als = ALS(rank = 20, maxIter = 7)
modelo = als.fit(df_avaliacao)

In [ ]:
# Visualizando o Affinity Score
modelo.userFactors.orderBy("id").collect()

In [ ]:
# Criando um dataset de teste com usuários e items para rating
df_teste = spSession.createDataFrame([(1001, 9003),(1001,9004),(1001,9005)], ["user", "item"])

In [ ]:
# Previsões  
# Quanto maior o Affinity Score, maior a probabilidade do usuário aceitar uma recomendação
previsoes = (modelo.transform(df_teste).collect())
previsoes

# Fim